## Setup

In [ ]:
%matplotlib inline
%load_ext autoreload
%autoreload 2
import os
import sys
import matplotlib.pyplot as plt
from matplotlib import animation, rc
from IPython.display import HTML
from matplotlib.pyplot import Rectangle

p = os.path.join(os.path.dirname('__file__'), '../..')
sys.path.append(p)
from common import *

In [ ]:
# Untar files into data directory
# tar zxvf bbox_images.tar.gz
DATA_DIR = '../../data/'
IMG_DIR = os.path.join(DATA_DIR, 'volleyball', 'images_subset')
metadata_fpath = os.path.join(DATA_DIR, 'volleyball_bbox_labels.csv')

In [ ]:
%ls {DATA_DIR}
%ls {IMG_DIR}

In [ ]:
# Remove 4 bad files
%rm {IMG_DIR}/._*

## Helpers

In [ ]:
def plot_img(arr, fs=(10,10), cmap='gray', title=None):
    plt.figure(figsize=fs)
    plt.imshow(arr, cmap=cmap)
    plt.title(title)
    plt.show()
    
def load_img(fpath):
    return plt.imread(fpath)

def load_cv2_img(fpath, w=None, h=None, colorspace=None):
    img = cv2.imread(img_fpath)
    if colorspace is not None:
        img = cv2.cvtColor(img, colorspace)
    if None not in [w,h]:
        img = cv2.resize(img, (w, h), interpolation=cv2.INTER_CUBIC)
    return img

def make_boxes(meta):
    boxes = {}
    for idx,row in meta.iterrows():
        box = json.loads(row.to_json())
        fname = row['filename']
        if fname in boxes:
            boxes[fname].append(box)
        else:
            boxes[fname] = [box]
    return boxes

In [ ]:
metadata = pd.read_csv(metadata_fpath)
metadata['label_name'] = 'ball'
metadata['label_id'] = 1
fnames = metadata['filename']
fpaths = [os.path.join(IMG_DIR, f) for f in fnames]
metadata['fpath'] = fpaths
GT_BOXES = make_boxes(metadata)

## Explore

In [ ]:
img_fpath = fpaths[random.randint(0,len(fpaths)-1)]
img = load_img(img_fpath)
img = load_cv2_img(img_fpath, 640, 360)
plot_img(img, fs=(15,12))

## Colorspace

Why do we convert?


Links
* https://www.learnopencv.com/color-spaces-in-opencv-cpp-python/
* https://docs.opencv.org/3.2.0/df/d9d/tutorial_py_colorspaces.html

### BGR

In [ ]:
bgr_img = load_cv2_img(img_fpath)

# Shape (w,h,c)
print(bgr_img.shape)

# Plot
plot_img(bgr_img, fs=(10,10))

# Channels
b,g,r = bgr_img[:,:,0], bgr_img[:,:,1], bgr_img[:,:,2]

plot_img(b, title='Blue')
plot_img(g, title='Green')
plot_img(r, title='Red')

### RGB

* Additive - combines Red, Green, Blue values
* 3 Channels correlated by amount of light hitting surface
* Problems
    * Mixes color (chrominance) )and intensity (luminance) information into a single value

In [ ]:
rgb_img = load_cv2_img(img_fpath, colorspace=cv2.COLOR_BGR2RGB)

# Plot
plot_img(rgb_img, fs=(10,10))

### HSV

* Hue (Dominant wavelength)
* Saturation (Purity / shades of the color)
* Value (Intensity)

Pros
* Only one channel needed to describe color (H)
* Best for color thresholding (why?)
* More robust to reflections on the floor

Cons
* Device dependent

![HSV]](https://edoras.sdsu.edu/doc/matlab/toolbox/images/hsvcone.gif)

In [ ]:
hsv_img = load_cv2_img(img_fpath, colorspace=cv2.COLOR_BGR2HSV)

# Plot
plot_img(hsv_img, fs=(10,10))

# Channels (Hue, Saturation, Value)
h,s,v = hsv_img[:,:,0], hsv_img[:,:,1], hsv_img[:,:,2]

plot_img(h, title='Hue')
plot_img(s, title='Saturation')
plot_img(v, title='Value')

In [ ]:
# Determine HSV value of specific color

blue = np.uint8([[[255,0,0 ]]])
hsv_blue = cv2.cvtColor(blue, cv2.COLOR_BGR2HSV)
print("Blue", hsv_blue)

green = np.uint8([[[0,255,0 ]]])
hsv_green = cv2.cvtColor(green, cv2.COLOR_BGR2HSV)
print("Green", hsv_green)

red = np.uint8([[[0,0,255]]])
hsv_red = cv2.cvtColor(red, cv2.COLOR_BGR2HSV)
print("Red", hsv_red)

## Color Thresholding

* https://docs.opencv.org/3.2.0/df/d9d/tutorial_py_colorspaces.html
* https://www.learnopencv.com/color-spaces-in-opencv-cpp-python/

### Find color of pixel

In [ ]:
def get_color_of_pixel(fpath, x, y, colorspace='BGR'):
    rgb_img = load_cv2_img(img_fpath, colorspace=cv2.COLOR_BGR2RGB)
    hsv_img = load_cv2_img(img_fpath, colorspace=cv2.COLOR_BGR2HSV)
    
    rgb_colors = rgb_img[y,x,:]
    hsv_colors = hsv_img[y,x,:]
    bgr_colors = np.copy(rgb_colors[::-1])
    print ("BGR:", bgr_colors)
    print ("RGB:", rgb_colors)
    print ("HSV:", hsv_colors)
    
    # Plot to visualize
    img = np.copy(rgb_img)
    img[y-5:y+5:,x-5:x+5,:] = 255
    img[y,x,:] = 0
    plot_img(img, fs=(18,18))
    
    if colorspace == 'BGR':
        return bgr_colors.tolist()
    if colorspace == 'RGB':
        return rgb_colors.tolist()
    return hsv_colors.tolist()

get_color_of_pixel(img_fpath, 100, 100, 'RGB')

In [ ]:
def get_hsv_value_of_bgr(bgr_color):
    print("BGR", bgr_color)
    bgr_color = np.uint8([[bgr_color]])
    hsv = cv2.cvtColor(bgr_color, cv2.COLOR_BGR2HSV)[0][0]
    print("HSV", hsv)
    return hsv

blue = [255,0,0 ]
_ = get_hsv_value_of_bgr(blue)

### Color Histograms

In [ ]:
bgr_img = load_cv2_img(img_fpath)

# Cv2 Histogram (faster)
hist = cv2.calcHist(images=[bgr_img], channels=[0], mask=None, histSize=[256], ranges=[0,256])

# Numpy Histogram (slower)
hist, bins = np.histogram(bgr_img.ravel(), 256, [0,256])

In [ ]:
# Plotting Histogram (all channels flattened)
def plot_hist(img, bins=256, title=None):
    plt.hist(img.ravel(), bins=bins, range=[0,256])
    plt.title(title)
    plt.show()

plot_hist(bgr_img)

In [ ]:
def plot_bgr_hist(bgr_img, bins=256, mask=None):
    # Mask let's you select for certain regions    
    color = ('b','g','r')
    for i,col in enumerate(color):
        histr = cv2.calcHist([bgr_img],[i],mask,[bins],[0,256])
        plt.plot(histr, color=col)
        plt.xlim([0,bins])
    plt.show()

plot_bgr_hist(bgr_img)

## Thresholding

In [ ]:
def threshold(img, color, thresh):
    """
    color = [b, g, r] or [r,b,g] or [h,s,v]
    thresh = margin allowed around color
    """
    min_color = np.array([color[0]-thresh, color[1]-thresh, color[2]-thresh])
    max_color = np.array([color[0]+thresh, color[1]+thresh, color[2]+thresh])
    min_color[min_color < 0] = 0
    max_color[max_color > 255] = 255
    print("Min", min_color)
    print("Max", max_color)
    
    mask = cv2.inRange(img, min_color, max_color)
    result = cv2.bitwise_and(img, img, mask=mask)
    return mask, result

### BGR Thresholding

In [ ]:
# Highlight the court
bgr_color = get_color_of_pixel(img_fpath, 700, 650, 'BGR')
bgr_img = load_cv2_img(img_fpath)
mask, result = threshold(bgr_img, bgr_color, 35)
plot_img(result)

In [ ]:
# Highlight floor around court (darker green)
bgr_color = get_color_of_pixel(img_fpath, 1100, 550, 'BGR')
bgr_img = load_cv2_img(img_fpath)
mask1, result1 = threshold(bgr_img, bgr_color, 60)
plot_img(result1)

In [ ]:
# Highlight floor around court (lighter green reflection)
bgr_color = get_color_of_pixel(img_fpath, 1130, 460, 'BGR')
bgr_img = load_cv2_img(img_fpath)
mask2, result2 = threshold(bgr_img, bgr_color, 50)
plot_img(result2)

### HSV Thresholding

In [ ]:
hsv_color = get_color_of_pixel(img_fpath, 700, 650, 'HSV')
hsv_img = load_cv2_img(img_fpath, colorspace=cv2.COLOR_BGR2HSV)
mask, result = threshold(hsv_img, hsv_color, 35)
plot_img(result)

In [ ]:
hsv_color = get_color_of_pixel(img_fpath, 1130, 460, 'HSV')
hsv_img = load_cv2_img(img_fpath, colorspace=cv2.COLOR_BGR2HSV)
mask, result = threshold(hsv_img, hsv_color, 55)
plot_img(result)

In [ ]:
hsv_color = get_color_of_pixel(img_fpath, 25, 550, 'HSV')
hsv_img = load_cv2_img(img_fpath, colorspace=cv2.COLOR_BGR2HSV)
mask, result = threshold(hsv_img, hsv_color, 55)
plot_img(result)

## Data Driven Thresholding

* https://gist.github.com/danielballan/ab5e28420ba1b24c5ad4

In [ ]:
json_fpath = os.path.join(DATA_DIR, 'volleyball_frame_00665.json')
img_fpath = os.path.join(DATA_DIR, 'volleyball_frame_00665.png')

BOX_COLORS = {
    'referee': 'black',
    'red_team': 'red',
    'blue_team': 'blue',
    'court-inner': 'green',
    'court_outer': 'white',
}

def plot_bbs_from_rectLabel_annos(fpath):
    bb_json = json.load(open(json_fpath, 'r'))
    img_fpath = os.path.join(IMG_DIR, bb_json['filename'])
    fig = plt.figure(figsize=(18,18))
    axes = plt.axes([0, 0.03, 1, 0.97])
    
    img = plt.imread(img_fpath)
    imgplot = axes.imshow(img)

    for box in bb_json['objects']:
        label = box['label']
        color = BOX_COLORS[label]
        coords = box['x_y_w_h']
        bb = Rectangle(
            (coords[0],coords[1]), 
            coords[2], coords[3],
            fill=False,
            edgecolor=color,
            linewidth=2)
        axes.add_patch(bb)
        
plot_bbs_from_rectLabel_annos(json_fpath)

In [ ]:
def get_img_crops_from_rectLabel_bbs(img, json_fpath):
    crops_dict = {}
    bb_json = json.load(open(json_fpath, 'r'))
    for box in bb_json['objects']:
        label = box['label']
        x,y,w,h = box['x_y_w_h']
        crop = img[y:y+h,x:x+w,:]
        if label not in crops_dict:
            crops_dict[label] = []
        crops_dict[label].append(crop)
    return crops_dict

bgr_img = load_cv2_img(img_fpath)
crops = get_img_crops_from_rectLabel_bbs(bgr_img, json_fpath)


In [ ]:
for crop in crops['red_team']:
    plot_img(crop)

In [ ]:
for crop in crops['court-inner']:
    plot_img(crop)

In [ ]:
# Create color histograms representing average values among samples
def get_flattened_channels(imgs):
    chans = np.empty(shape=(1,3))
    for img in imgs:
        h,w,c = img.shape
        reshaped = img.reshape((h*w, c))
        chans = np.concatenate([chans, reshaped], axis=0)
        #print(reshaped.shape, chans.shape)
    return chans

In [ ]:
# BGR Histograms
bgr_img = load_cv2_img(img_fpath)
crops = get_img_crops_from_rectLabel_bbs(bgr_img, json_fpath)

hists = {}
for label in crops.keys():
    chans = get_flattened_channels(crops[label])
    bgr = ('b','g','r')
    hists[label] = {
        'b':None,
        'g':None,
        'r':None
    }
    for i in range(len(chans[0])):
        hist, bins = np.histogram(chans[:,i], 50, [0,256])
        hists[label][bgr[i]] = hist
        plot_hist(chans[:,i], bins=50, title=label + ' ' + bgr[i])

In [ ]:
# HSV Histograms
hsv_img = load_cv2_img(img_fpath, colorspace=cv2.COLOR_BGR2HSV)
crops = get_img_crops_from_rectLabel_bbs(hsv_img, json_fpath)

hists = {}
for label in crops.keys():
    chans = get_flattened_channels(crops[label])
    channels = ('h','s','v')
    hists[label] = {c:None for c in bgr}
    for i in range(len(chans[0])):
        hist, bins = np.histogram(chans[:,i], 50, [0,256])
        hists[label][bgr[i]] = hist
        plot_hist(chans[:,i], bins=50, title=label + ' ' + channels[i])

### RGB Threshold

In [ ]:
"""
court inner
B = 75 - 125
G = 90 - 135
R = 175 - 220
"""
def threshold(img, color, thresh, sigma=1.0):
    """
    color = [b, g, r] or [r,b,g] or [h,s,v]
    thresh = [b,g,r] margin allowed around color (1 per channel)
    """
    thresh = np.array(thresh) * sigma
    min_color = np.array([color[0]-thresh[0], color[1]-thresh[1], color[2]-thresh[2]])
    max_color = np.array([color[0]+thresh[0], color[1]+thresh[1], color[2]+thresh[2]])
    min_color[min_color < 0] = 0
    max_color[max_color > 255] = 255
    print("Min", min_color)
    print("Max", max_color)
    
    mask = cv2.inRange(img, min_color, max_color)
    result = cv2.bitwise_and(img, img, mask=mask)
    return mask, result

In [ ]:
# RGB

img_fpath = os.path.join(IMG_DIR, 'volleyball_frame_00665.png')
img = load_cv2_img(img_fpath, colorspace=cv2.COLOR_BGR2RGB)
plot_img(img, fs=(14,14), title="Original")

r = 175 + 220 // 2
g = 90 + 135 // 2
b = 75 + 125 // 2
color = (
    r,g,b
)
thresh = (
    220 - 175 // 2,
    135 - 90 // 2,
    125 - 75 // 2
)
mask, result = threshold(img, color, thresh, sigma=.9)
plot_img(result, fs=(14,14), title="Thresholded")

In [ ]:
# HSV

img_fpath = os.path.join(IMG_DIR, 'volleyball_frame_00665.png')
img = load_cv2_img(img_fpath, colorspace=cv2.COLOR_BGR2HSV)
plot_img(img, fs=(14,14), title="Original")
"""
H 0 - 10
S = 100 - 150
V = 175 - 220
"""
color = (
    5,
    50,
    45
)
thresh = (
    6,
    256,
    256
)
mask, result = threshold(img, color, thresh, sigma=1)
plot_img(result, fs=(14,14), title="Thresholded")

## Erosion and Dilation

* https://docs.opencv.org/3.0-beta/doc/py_tutorials/py_imgproc/py_morphological_ops/py_morphological_ops.html
* https://www.geeksforgeeks.org/erosion-dilation-images-using-opencv-python/
* Morphological operation
* Goals:
    * Remove noise
    * Join nearby elements

### Erosion

* Erodes away boundaries of objects
* Kernel slides through and returns 1 only if all neighbors are also 1
* Thickness or size of object decreases (boundary pixels are disgarded
* Use cases
    * Removing noise around edges
    * Detaching two connected objects

In [ ]:
img_fpath = os.path.join(IMG_DIR, 'volleyball_frame_00665.png')

In [ ]:
# RGB
img = load_cv2_img(img_fpath, colorspace=cv2.COLOR_BGR2RGB)
plot_img(img, fs=(18,18), title="Original")

kernel = cv2.getStructuringElement(cv2.MORPH_ERODE, (3,3))
dilation = cv2.erode(img, kernel,iterations = 2)
plot_img(dilation, fs=(18,18), title="Erosion")

### Dilation

* Opposite of erosion
* Expands object around edges
* Useful for joining broken parts of an image
* Typically we erode first (to remove noise around edges), then dilate to increase size of object
* Use cases
    * Increasing side of object after erosion
    * Joining broken parts of an object
    
![Ellipse](https://study.com/cimages/multimages/16/semimajax2.jpg)

In [ ]:
# RGB
img = load_cv2_img(img_fpath, colorspace=cv2.COLOR_BGR2RGB)
plot_img(img, fs=(18,18), title="Original")

kernel = cv2.getStructuringElement(cv2.MORPH_ELLIPSE, (3,3))
dilation = cv2.dilate(img, kernel,iterations = 2)
plot_img(dilation, fs=(18,18), title="Dilation")

### Erosion + Dilation

In [ ]:
# RGB
img = load_cv2_img(img_fpath, colorspace=cv2.COLOR_BGR2RGB)
plot_img(img, fs=(14,14), title="Original")

kernel = cv2.getStructuringElement(cv2.MORPH_ELLIPSE, (3,3))
erosion = cv2.erode(img, kernel,iterations = 1)
plot_img(erosion, fs=(14,14), title="Erosion")

kernel = cv2.getStructuringElement(cv2.MORPH_ELLIPSE, (3,3))
dilation = cv2.dilate(img, kernel,iterations = 1)
plot_img(dilation, fs=(14,14), title="Dilation")

combined = cv2.dilate(erosion, kernel, iterations = 1)
plot_img(combined, fs=(14,14), title="Erosion + Dilation")

### HSV Erosion + Dilation

In [ ]:
# RGB
img = load_cv2_img(img_fpath, colorspace=cv2.COLOR_BGR2HSV)
plot_img(img, fs=(14,14), title="Original")

kernel = cv2.getStructuringElement(cv2.MORPH_ELLIPSE, (3,3))
erosion = cv2.erode(img, kernel,iterations = 3)
plot_img(erosion, fs=(14,14), title="Erosion")

kernel = cv2.getStructuringElement(cv2.MORPH_ELLIPSE, (3,3))
dilation = cv2.dilate(img, kernel,iterations = 1)
plot_img(dilation, fs=(14,14), title="Dilation")

combined = cv2.dilate(erosion, kernel, iterations = 1)
plot_img(combined, fs=(14,14), title="Erosion + Dilation")

## Canny Edge Detector

In [ ]:
img_fpath = os.path.join(IMG_DIR, 'volleyball_frame_00665.png')
img = load_cv2_img(img_fpath, colorspace=cv2.COLOR_BGR2RGB)
plot_img(img, fs=(14,14), title="Original")

gray = cv2.cvtColor(img, cv2.COLOR_RGB2GRAY)
gray = cv2.GaussianBlur(gray, ksize=(5,5), sigmaX=3)
plot_img(gray, fs=(14,14), title="Grayscale")

# use Canny edge detector to find edges in the image.  The thresholds determine how
# weak or strong an edge will be detected.  These can be tweaked.
lower_threshold = 50
upper_threshold = 50
edges = cv2.Canny(gray, lower_threshold, upper_threshold)
plot_img(edges, fs=(14,14))
print(edges.shape)

# Mask top of image
edges[:400,:] = 0
edges[:,1150:] = 0
plot_img(edges, fs=(14,14))

### Erosion, Dilation, Gaussian, Canny

In [ ]:
# RGB
img = load_cv2_img(img_fpath, colorspace=cv2.COLOR_BGR2RGB)
plot_img(img, fs=(14,14), title="Original")

kernel = cv2.getStructuringElement(cv2.MORPH_ELLIPSE, (3,3))
erosion = cv2.erode(img, kernel,iterations = 1)
plot_img(erosion, fs=(14,14), title="Erosion")

kernel = cv2.getStructuringElement(cv2.MORPH_ELLIPSE, (4,4))
dilation = cv2.dilate(img, kernel,iterations = 2)
plot_img(dilation, fs=(14,14), title="Dilation")

img = cv2.dilate(erosion, kernel, iterations = 2)
plot_img(img, fs=(14,14), title="Erosion + Dilation")

gray = cv2.cvtColor(img, cv2.COLOR_RGB2GRAY)
gray = cv2.GaussianBlur(gray, ksize=(5,5), sigmaX=3)
plot_img(gray, fs=(14,14), title="Grayscale")

# use Canny edge detector to find edges in the image.  The thresholds determine how
# weak or strong an edge will be detected.  These can be tweaked.
lower_threshold = 25
upper_threshold = 25
edges = cv2.Canny(gray, lower_threshold, upper_threshold)
plot_img(edges, fs=(14,14))

# Mask top of image
edges[:425,:] = 0
edges[700:,:] = 0
edges[:,1150:] = 0
plot_img(edges, fs=(14,14))

In [ ]:
# Hough Transform - Lines

# detect lines in the image.  This is where the real work is done.  Higher threshold
# means a line needs to be stronger to be detected, so again, this can be tweaked.
thresh = 125
lines = cv2.HoughLines(edges, 1, np.pi / 180, thresh)
img = load_cv2_img(img_fpath, colorspace=cv2.COLOR_BGR2RGB)

# convert each line to coordinates back in the original image
for line in lines:
    for rho, theta in line:
        a = np.cos(theta)
        b = np.sin(theta)
        x0 = a * rho
        y0 = b * rho
        x1 = int(x0 + 1000 * -b)
        y1 = int(y0 + 1000 * a)
        x2 = int(x0 - 1000 * -b)
        y2 = int(y0 - 1000 * a)

        # draw each line on the image
        cv2.line(img, pt1=(x1, y1), pt2=(x2, y2), color=(0, 0, 255), thickness=3)

# write the image to disk
plot_img(img, fs=(18,18))

### HSV - Erosion, Dilation, Blur, Canny

In [ ]:
# HSV
img = load_cv2_img(img_fpath, colorspace=cv2.COLOR_BGR2HSV)
plot_img(img, fs=(14,14), title="Original")

kernel = cv2.getStructuringElement(cv2.MORPH_ELLIPSE, (3,3))
erosion = cv2.erode(img, kernel,iterations = 1)
plot_img(erosion, fs=(14,14), title="Erosion")

kernel = cv2.getStructuringElement(cv2.MORPH_ELLIPSE, (3,3))
dilation = cv2.dilate(img, kernel,iterations = 2)
plot_img(dilation, fs=(14,14), title="Dilation")

img = cv2.dilate(img, kernel, iterations = 2)
plot_img(img, fs=(14,14), title="Erosion + Dilation")

gray = cv2.cvtColor(img, cv2.COLOR_RGB2GRAY)
gray = cv2.GaussianBlur(gray, ksize=(5,5), sigmaX=3)
plot_img(gray, fs=(14,14), title="Grayscale")

# use Canny edge detector to find edges in the image.  The thresholds determine how
# weak or strong an edge will be detected.  These can be tweaked.
lower_threshold = 25
upper_threshold = 125
edges = cv2.Canny(gray, lower_threshold, upper_threshold)
plot_img(edges, fs=(14,14))

# Mask top of image
edges[:425,:] = 0
edges[700:,:] = 0
edges[:,1150:] = 0
plot_img(edges, fs=(14,14))

## Hough Transform

In [ ]:
# Hough Transform - Lines

# detect lines in the image.  This is where the real work is done.  Higher threshold
# means a line needs to be stronger to be detected, so again, this can be tweaked.
thresh = 175
lines = cv2.HoughLines(edges, 1, np.pi / 180, thresh)
img = load_cv2_img(img_fpath, colorspace=cv2.COLOR_BGR2RGB)

# convert each line to coordinates back in the original image
for line in lines:
    for rho, theta in line:
        a = np.cos(theta)
        b = np.sin(theta)
        x0 = a * rho
        y0 = b * rho
        x1 = int(x0 + 1000 * -b)
        y1 = int(y0 + 1000 * a)
        x2 = int(x0 - 1000 * -b)
        y2 = int(y0 - 1000 * a)

        # draw each line on the image
        cv2.line(img, pt1=(x1, y1), pt2=(x2, y2), color=(0, 0, 255), thickness=3)

# write the image to disk
plot_img(img, fs=(18,18))

### RGB, Color Thresh, Canny, Hough

In [ ]:
img_fpath = os.path.join(IMG_DIR, 'volleyball_frame_00665.png')
img = load_cv2_img(img_fpath, colorspace=cv2.COLOR_BGR2RGB)
plot_img(img, fs=(14,14), title="Original")

r = 175 + 220 // 2
g = 90 + 135 // 2
b = 75 + 125 // 2
color = (
    r,g,b
)
thresh = (
    220 - 175 // 2,
    135 - 90 // 2,
    125 - 75 // 2
)
mask, img = threshold(img, color, thresh, sigma=.9)
plot_img(img, fs=(14,14), title="Thresholded")

In [ ]:
# RGB
kernel = cv2.getStructuringElement(cv2.MORPH_ELLIPSE, (3,3))
erosion = cv2.erode(img, kernel,iterations = 1)
plot_img(erosion, fs=(14,14), title="Erosion")

kernel = cv2.getStructuringElement(cv2.MORPH_ELLIPSE, (4,4))
dilation = cv2.dilate(img, kernel,iterations = 1)
plot_img(dilation, fs=(14,14), title="Dilation")

img = cv2.dilate(erosion, kernel, iterations = 1)
plot_img(img, fs=(14,14), title="Erosion + Dilation")

gray = cv2.cvtColor(img, cv2.COLOR_RGB2GRAY)
gray = cv2.GaussianBlur(gray, ksize=(5,5), sigmaX=3)
plot_img(gray, fs=(14,14), title="Grayscale")

# use Canny edge detector to find edges in the image.  The thresholds determine how
# weak or strong an edge will be detected.  These can be tweaked.
lower_threshold = 100
upper_threshold = 200
edges = cv2.Canny(gray, lower_threshold, upper_threshold)
plot_img(edges, fs=(14,14))

# Mask top of image
# edges[:425,:] = 0
# edges[700:,:] = 0
# edges[:,1150:] = 0
plot_img(edges, fs=(14,14))

In [ ]:
# Hough Transform - Lines

# detect lines in the image.  This is where the real work is done.  Higher threshold
# means a line needs to be stronger to be detected, so again, this can be tweaked.
thresh = 225
lines = cv2.HoughLines(edges, 1, np.pi / 180, thresh)
img = load_cv2_img(img_fpath, colorspace=cv2.COLOR_BGR2RGB)

# convert each line to coordinates back in the original image
for line in lines:
    for rho, theta in line:
        a = np.cos(theta)
        b = np.sin(theta)
        x0 = a * rho
        y0 = b * rho
        x1 = int(x0 + 1000 * -b)
        y1 = int(y0 + 1000 * a)
        x2 = int(x0 - 1000 * -b)
        y2 = int(y0 - 1000 * a)

        # draw each line on the image
        cv2.line(img, pt1=(x1, y1), pt2=(x2, y2), color=(0, 0, 255), thickness=3)

# write the image to disk
plot_img(img, fs=(18,18))

### HSV Color Thresh, Canny, Hough

In [ ]:
# HSV

img_fpath = os.path.join(IMG_DIR, 'volleyball_frame_00665.png')
img = load_cv2_img(img_fpath, colorspace=cv2.COLOR_BGR2HSV)
#plot_img(img, fs=(14,14), title="Original")
"""
H 0 - 10
S = 100 - 150
V = 175 - 220
"""
color = (
    5,
    50,
    45
)
thresh = (
    7,
    256,
    256
)
mask, img = threshold(img, color, thresh, sigma=1)
#plot_img(img, fs=(14,14), title="Thresholded")

In [ ]:
# RGB
kernel = cv2.getStructuringElement(cv2.MORPH_ELLIPSE, (3,3))
erosion = cv2.erode(img, kernel,iterations = 1)
plot_img(erosion, fs=(14,14), title="Erosion")

kernel = cv2.getStructuringElement(cv2.MORPH_ELLIPSE, (3,3))
dilation = cv2.dilate(img, kernel,iterations = 1)
plot_img(dilation, fs=(14,14), title="Dilation")

img = cv2.dilate(img, kernel, iterations = 2)
plot_img(img, fs=(14,14), title="Erosion + Dilation")

gray = cv2.cvtColor(img, cv2.COLOR_RGB2GRAY)
gray = cv2.GaussianBlur(gray, ksize=(5,5), sigmaX=3)
plot_img(gray, fs=(14,14), title="Grayscale")

# use Canny edge detector to find edges in the image.  The thresholds determine how
# weak or strong an edge will be detected.  These can be tweaked.
lower_threshold = 100
upper_threshold = 200
edges = cv2.Canny(gray, threshold1=lower_threshold, threshold2=upper_threshold, apertureSize=3)
plot_img(edges, fs=(14,14))

# Mask top of image
plot_img(edges, fs=(14,14))

In [ ]:
img.shape

### HoughLinesP

* Probabalistic Hough Lines

In [ ]:
# Hough Transform - Lines

x_delta = 100
y_delta = 100
min_line_length = 400
max_line_gap = 2000
thresh = 100
#minLineLength - Minimum length of line
#maxLineGap - Maximum allowed gap between line segments to treat them as single line.
lines = cv2.HoughLinesP(edges, rho=1, theta=np.pi/180,
                        threshold=thresh, maxLineGap=max_line_gap, 
                        minLineLength=min_line_length)
line_coords = []

for line in lines:
    x1,y1,x2,y2 = line[0]
    print(x1,y1,x2,y2)
    if abs(x2 - x1) < x_delta:
        orient = 'vertical'
    elif abs(y2 - y1) < y_delta:
        orient = 'horizontal'
    else:
        orient = 'other' #None
        print('skipping line', x1,y1,x2,y2)
    if orient is not None:
        line_coords.append([x1,y1,x2,y2])
        img = load_cv2_img(img_fpath, colorspace=cv2.COLOR_BGR2RGB)
        cv2.line(img, pt1=(x1, y1), pt2=(x2, y2), color=(0, 0, 255), thickness=3)
        plot_img(img, fs=(18,18), title=coords)

### HoughLines

In [ ]:
thresh = 125
lines = cv2.HoughLines(edges, 1, np.pi / 180, thresh)
img = load_cv2_img(img_fpath, colorspace=cv2.COLOR_BGR2RGB)

line_coords = []

max_y,max_x,chan = img.shape
# convert each line to coordinates back in the original image
for line in lines:
    for p, theta in line:
        # Stores the value of cos(theta) in a
        a = np.cos(theta)
        
        # Stores the value of sin(theta) in b
        b = np.sin(theta)
        
        # x0 stores the value p*cos(theta)
        x0 = a * p
        
        # y0 stores the value p*sin(theta)
        y0 = b * p
        
        # x1,x2,y1,y2 store rounded off values
        x1 = int(x0 + 1000 * (-b))
        y1 = int(y0 + 1000 * (a))
        x2 = int(x0 - 1000 * (-b))
        y2 = int(y0 - 1000 * (a))
        
        slope = (y2-y1)/(x2-x1)
        # Filter out lines completely above the court
        if y2 > 400 or y1 > 400:
            # Label Horizontal / Vertical
            if abs(slope) > 1:
                orient = 'vertical'
            else:
                orient = 'horizontal'
            coords = [x1,y1,x2,y2,p,theta,slope,orient]
            line_coords.append(coords)
            img = load_cv2_img(img_fpath, colorspace=cv2.COLOR_BGR2RGB)
            cv2.line(img, pt1=(x1, y1), pt2=(x2, y2), color=(0, 0, 255), thickness=3)
            plot_img(img, fs=(18,18), title=(x1,x2,y1,y2,p,theta,orient))

In [ ]:
# What the HoughTransform returns (p, and theta)
for line in lines:
    for p, radians in line:
        print(line[0], p, radians)

In [ ]:
# Label Horizontal / Vertical
for line in line_coords:
    if abs(line[2] - line[0]) < 200:
        line.append('vertical')
    elif abs(line[3] - line[1]) < 200:
        line.append('horizontal')

In [ ]:
# x1,y2,x2,y2,p,theta,orientation
line_coords

### Cleaning Up Hough Lines

* https://campushippo.com/lessons/detect-highway-lane-lines-with-opencv-and-python-21438a3e2
* How to clean up the lines, extend them, and average nearby lines?
* https://stackoverflow.com/questions/44449871/fine-tuning-hough-line-function-parameters-opencv
* Since the lines are mostly vertical and horizontal, you can easily split the lines based on their position. If the two y-coordinates of one line are near each other, then the line is mostly horizontal. If the two x-coordinates are near each other, then the line is mostly vertical. So you can segment your lines into vertical lines and horizontal lines that way.
* https://alyssaq.github.io/2014/understanding-hough-transform/

ρ = x cos θ + y sin θ

where:
ρ (rho) = distance from origin to the line. [-max_dist to max_dist].
          max_dist is the diagonal length of the image.  
θ = angle from origin to the line. [-90° to 90°]

## Hough Transform


* https://docs.opencv.org/3.0-beta/doc/py_tutorials/py_imgproc/py_houghlines/py_houghlines.html
* https://docs.opencv.org/3.3.1/d9/db0/tutorial_hough_lines.html
* rho = 
* Returns lines in parametric (polar coordinates) form (p, theta)
    * p = x cos(theta) + y sin(theta)
    * p = perpendicular distance from origin (0,0) - top left corner
    * theta = angle formed by p and horizontal axis (counter clockwise) (radians)
    * r is measured in pixels and 0 is measured in radians.
![Houghlines](https://docs.opencv.org/3.0-beta/_images/houghlines1.svg)
![hough](http://cdncontribute.geeksforgeeks.org/wp-content/uploads/Hough_transform_diagram.png)

* https://www.geeksforgeeks.org/line-detection-python-opencv-houghline-method/
* Applications of Hough transform:

    * It is used to isolate features of a particular shape within an image.
    * Tolerant of gaps in feature boundary descriptions and is relatively unaffected by noise.
    * Used extensively in barcode scanning, verification and recognition
* Steps
    * First parameter, Input image should be a binary image, so apply threshold edge detection before finding applying hough transform.
    * Second and third parameters are r and θ(theta) accuracies respectively.
    * Fourth argument is the threshold, which means minimum vote it should get for it to be considered as a line.
    * Remember, number of votes depend upon number of points on the line. So it represents the minimum length of line that should be detected.

## Harris Corners

In [ ]:
img = load_cv2_img(img_fpath)
gray = cv2.GaussianBlur(gray, ksize=(5,5), sigmaX=3)
gray = cv2.cvtColor(img,cv2.COLOR_BGR2GRAY)
gray = np.float32(gray)

dst = cv2.cornerHarris(gray,2,3,0.04)

#result is dilated for marking the corners, not important
dst = cv2.dilate(dst,None)

# Threshold for an optimal value, it may vary depending on the image.
img[dst>0.05*dst.max()]=[0,0,255]

plot_img(img, fs=(20,15))

In [ ]:
img = load_cv2_img(img_fpath)
#gray = cv2.GaussianBlur(gray, ksize=(5,5), sigmaX=3)
gray = cv2.cvtColor(img,cv2.COLOR_BGR2GRAY)
gray = np.float32(gray)

dst = cv2.cornerHarris(gray,2,3,0.04)
dst = cv2.dilate(dst,None)
ret, dst = cv2.threshold(dst,0.01*dst.max(),255,0)
dst = np.uint8(dst)

# find centroids
ret, labels, stats, centroids = cv2.connectedComponentsWithStats(dst)

# define the criteria to stop and refine the corners
criteria = (cv2.TERM_CRITERIA_EPS + cv2.TERM_CRITERIA_MAX_ITER, 100, 0.001)
corners = cv2.cornerSubPix(gray,np.float32(centroids),(5,5),(-1,-1),criteria)

# Now draw them
res = np.hstack((centroids,corners))
res = np.int0(res)
img[res[:,1],res[:,0]]=[0,0,255]
img[res[:,3],res[:,2]] = [0,255,0]

plot_img(img, fs=(20,15))

## Contours

In [ ]:
img = load_cv2_img(img_fpath)
gray = cv2.GaussianBlur(gray, ksize=(5,5), sigmaX=3)
gray = cv2.cvtColor(img,cv2.COLOR_BGR2GRAY)

ret,thresh = cv2.threshold(gray, 127, 255, 0)
image, contours, hierarchy = cv2.findContours(thresh,cv2.RETR_TREE,cv2.CHAIN_APPROX_SIMPLE)
img = cv2.drawContours(img, contours, -1, (0,255,0), 3)
plot_img(img, fs=(20,15))

In [ ]:
# https://docs.opencv.org/3.3.1/dd/d49/tutorial_py_contour_features.html



### Histograms

In [ ]:
img = cv2.imread(img_fpath)

In [ ]:
# Loading Histogram

# OpenCV - faster
hist = cv2.calcHist(images=[img], channels=[0], mask=None, histSize=[256], ranges=[0,256])

# Numpy
hist, bins = np.histogram(img.ravel(), 256, [0,256])

In [ ]:
# Plotting Histogram
plt.hist(img.ravel(), 256, [0,256])
plt.show()

In [ ]:
# Color histogram

def plot_color_hist(img, bins=256, mask=None):
    # Mask let's you select for certain regions
    
    color = ('b','g','r')
    for i,col in enumerate(color):
        histr = cv2.calcHist([img],[i],mask,[bins],[0,256])
        plt.plot(histr, color=col)
        plt.xlim([0,bins])
    plt.show()

In [ ]:
## Applying a Mask

mask = np.zeros(img.shape[:2], np.uint8)
mask[400:700, 100:1100] = 255
masked_img = cv2.bitwise_and(img, img, mask=mask)

plot_img(load_cv2_img(img_fpath))
plot_img(mask)
plot_img(masked_img)

plot_color_hist(img, bins=30)
plot_color_hist(img, bins=30, mask=mask)

In [ ]:
# HSV Histogram - Hue holds the color information nicely
img = cv2.imread(img_fpath)
hsv = cv2.cvtColor(img, cv2.COLOR_BGR2HSV)
hist = cv2.calcHist(images=[hsv], channels=[0], mask=None, histSize=[359], ranges=[0,359])
plt.plot(hist)

In [ ]:
hsv.shape

In [ ]:
# Color Quantization
# https://docs.opencv.org/3.0-beta/doc/py_tutorials/py_ml/py_kmeans/py_kmeans_opencv/py_kmeans_opencv.html

img = cv2.imread(img_fpath)
hsv = cv2.cvtColor(img, cv2.COLOR_BGR2HSV)
gray = cv2.cvtColor(img, cv2.COLOR_BGR2GRAY)
Z = img.reshape((-1,3))

# convert to np.float32
Z = np.float32(Z)

# define criteria, number of clusters(K) and apply kmeans()
criteria = (cv2.TERM_CRITERIA_EPS + cv2.TERM_CRITERIA_MAX_ITER, 10, 1.0)
K = 6
ret, label, center = cv2.kmeans(Z,K,None,criteria,10,cv2.KMEANS_RANDOM_CENTERS)

# Now convert back into uint8, and make original image
center = np.uint8(center)
res = center[label.flatten()]
res2 = res.reshape((img.shape))

plot_img(res2, fs=(12,8))

In [ ]:
plot_color_hist(res2)

In [ ]:
h,w,c = img.shape
minarea = h * w / 10
gray = cv2.GaussianBlur(res2, ksize=(5,5), sigmaX=3)
gray = cv2.cvtColor(gray, cv2.COLOR_BGR2GRAY)

ret,thresh = cv2.threshold(gray, 127, 255, 0)
image, contours, hierarchy = cv2.findContours(thresh,cv2.RETR_TREE,cv2.CHAIN_APPROX_SIMPLE)
img = cv2.drawContours(img, contours, -1, (0,255,0), 3)

In [ ]:
for cnt in contours:
    if cv2.contourArea(cnt) < minarea:
        img = cv2.drawContours(img, [cnt], -1, (0,0,0), 3)


In [ ]:
plot_img(img, fs=(20,15))

In [ ]:
# Segment with Color Range

# Find the dominant color in the target region with masking
mask = np.zeros(img.shape[:2], np.uint8)
mask[600:650, 100:150] = 255
masked_img = cv2.bitwise_and(img, img, mask=mask)

plot_img(load_cv2_img(img_fpath))
plot_img(mask)
plot_img(masked_img)

plot_color_hist(img, bins=256)
plot_color_hist(img, bins=256, mask=mask)

In [ ]:
img = cv2.imread(img_fpath)
img = cv2.GaussianBlur(img, ksize=(5,5), sigmaX=3)
hsv = cv2.cvtColor(img, cv2.COLOR_BGR2HSV)

court_color_bgr = np.uint8([[[75,100,190]]])
hsv_court_color = cv2.cvtColor(court_color_bgr, cv2.COLOR_BGR2HSV)
hue = hsv_court_color[0][0][0]
hue

In [ ]:
# define range of red color in HSV
lower = np.array([hue-5,10,10])
upper = np.array([hue+20,255,255])

In [ ]:
# Threshold the HSV image to get only blue colors
mask = cv2.inRange(hsv, lower, upper)
 
# Bitwise-AND mask and original image
masked_img = cv2.bitwise_and(img, img, mask=mask)

plot_img(load_cv2_img(img_fpath), fs=(12,8))
plot_img(mask, fs=(12,8))
plot_img(masked_img, fs=(12,8))

In [ ]:
gray = cv2.cvtColor(masked_img, cv2.COLOR_BGR2GRAY)
ret,thresh = cv2.threshold(gray, 127, 255, 0)
image, contours, hierarchy = cv2.findContours(thresh,cv2.RETR_TREE,cv2.CHAIN_APPROX_SIMPLE)
img = cv2.drawContours(img, contours, -1, (0,255,0), 3)

In [ ]:
plot_img(img, fs=(12,8))

In [ ]:
# define range of red color in HSV
lower = np.array([110,50,50])
upper = np.array([130,255,255])

# Threshold the HSV image to get only blue colors
mask = cv2.inRange(hsv, lower, upper)
 
# Bitwise-AND mask and original image
masked_img = cv2.bitwise_and(img, img, mask=mask)

plot_img(load_cv2_img(img_fpath))
plot_img(mask)
plot_img(masked_img)

In [ ]:
h,w,c = img.shape
minarea = h * w / 10
img = cv2.imread(img_fpath)
hsv = cv2.cvtColor(img, cv2.COLOR_BGR2HSV)
gray = cv2.cvtColor(img, cv2.COLOR_BGR2GRAY)
gray = cv2.GaussianBlur(gray, ksize=(5,5), sigmaX=2)
ret, thresh = cv2.threshold(gray, 127, 255, 0)
image, contours, hierarchy = cv2.findContours(thresh,cv2.RETR_TREE,cv2.CHAIN_APPROX_SIMPLE)

In [ ]:
cnts = []
for cnt in contours:
    if cv2.contourArea(cnt) > minarea:
        cnts.append(cnt)
len(cnts)

### Animation

In [ ]:
# Got it working!
fig = plt.figure()
axes = plt.axes([0, 0.03, 1, 0.97])

img = plt.imread(fpaths[0])
imgplot = axes.imshow(img, animated=True)

In [ ]:
all_boxes = GT_BOXES
def init():
    return (imgplot,)

def animate_w_boxes(fname):
    fpath = os.path.join(IMG_DIR, fname)
    img = plt.imread(fpath)
    imgplot.set_array(img)
    
    # Remove old boxes
    for p in axes.patches:
        p.remove()
    boxes = all_boxes[fname]
    for box in boxes:
        width = box['x2'] - box['x1']
        height = box['y2'] - box['y1']
        bb = Rectangle(
            (box['x1'],box['y1']), 
            width, height,
            fill=False,
            edgecolor="red")
        axes.add_patch(bb)    
    return (imgplot,)

In [ ]:
# call the animator. blit=True means only re-draw the parts that have changed.
anim = animation.FuncAnimation(fig, animate_w_boxes, init_func=init,
                               frames=fnames, interval=100, blit=True)

In [ ]:
# ~1 minute to general
HTML(anim.to_html5_video())

### Links

* https://towardsdatascience.com/finding-lane-lines-on-the-road-30cf016a1165
* https://towardsdatascience.com/how-to-train-your-own-object-detector-with-tensorflows-object-detector-api-bec72ecfe1d9
* https://github.com/christopher5106/FastAnnotationTool
* http://androidkt.com/train-object-detection/
* https://github.com/tensorflow/models/blob/master/research/object_detection/g3doc/using_your_own_dataset.md
* https://towardsdatascience.com/building-a-real-time-object-recognition-app-with-tensorflow-and-opencv-b7a2b4ebdc32